# Random Matrix Theory - Cavity method

Here we obtain the spectral density $\rho(\lambda)$ by starting from the following recursive expression for the cavity precisions

\begin{equation}
    \omega^{(j)}_{k} = i(\lambda - i \epsilon) + \sum_{\ell \in \partial k \backslash j} \frac{M^{2}_{k \ell}}{\omega^{(k)}_{\ell}}, \tag{1}
\end{equation}

then computing the marginal precisions through the equation

\begin{equation}
    \omega_{j} = i(\lambda - i \epsilon) + \sum_{k \in \partial j} \frac{M^{2}_{k j}}{\omega^{(j)}_{k}}, \tag{2}
\end{equation}

and finally getting the value of the spectral density via

\begin{equation}
    \rho(\lambda) = \lim_{\epsilon \to 0^{+}} \frac{1}{\pi N} \sum_{j = 1}^{N} \text{Im}\left[\frac{i}{\omega_{j}}\right].  \tag{3}
\end{equation}


## Algorithm

### Inputs

* Size of the matrix $N$.
* Mean connectivity $c$.
* Values of $\lambda$ and $\epsilon$.
* Maximum number of iterations per value of $\lambda$.
* Minimum absolute tolerance, i.e., maximum change allowed to say that we have achieved the fixed point.

### Ouputs

* Value of $\rho(\lambda)$.

### Procedure

1. Create instance of matrix $\mathbf{M}$ ($M_{i j} = A_{i j} J_{ij}$). 
    1. Create a random regular graph of size $N$ and mean connectivity $c$, and extract the adjacency matrix $\mathbf{A}$.
        
    1. Define normal distribution from $\mu$ and $\sigma$, and from this, draw the elements for the symmetric matrix $\mathbf{J}$. 
        
    1. Matrix $\mathbf{M}$ is given by the element-wise product of $\mathbf{A}$ and $\mathbf{J}$.
    
1. To avoid having to access different parts of the matrix $\mathbf{M}$, create two matrices with the elements of $\mathbf{M}$ ordered by neighbors, and an array with the positions of the cavity precisions for quick access:

    1. A matrix of shape $(c-1) \times (N \cdot c)$, that stores the values of $M^{2}_{k \ell}$ relevant for the recursive equation of each $\omega^{(j)}_{k}$.
    
    1. A matrix of shape $c \times N$ that stores the relevant values of $M^{2}_{k j}$ for the marginal precisions $\omega_{j}$.
    
    1. An array with the positions of the relevant cavity precisions $\omega^{(j)}_{k}$ in the order in which they appear in the recursive equations.

1. Update the values of the cavity precisions $\omega^{(j)}_{k}$.
    1. Keep track of the biggest absolute change $\Delta \omega$ after each update.
    
    1. After updating every cavity precision once, check if the maximum change lies below the tolerance. If it doesn't, iterate again; else, finish iterating. Repeat process for a specified maximum of iterations.
    
1. Compute the marginal precisions.
    1. Reshape the cavity precisions to be the same shape as the matrix containing the relevant values of $M$. 
    1. Perform the element-wise division $M^{2}_{k j} \; / \; \omega^{(j)}_{k}$.
    1. Sum along each column.
    1. Sum $i(\lambda - i \epsilon)$ to every element of the resulting vector.

1. Compute the spectral density.
    1. For each marginal precision (the resulting vector of the previous step), take the element-wise operation $i / \omega_{j}$.
    1. Sum the entire vector.
    1. Take the imaginary part.
    1. Divide by $(\pi N)$.

## 1. Load packages and define functions

In [ ]:
using Distributions  # Define normal distributions with given mean and standard deviation.
using Random  # Generate random numbers from given distributions.
using LightGraphs  # Generate Random Regular Graphs (in particular, adjacency matrices).
using LinearAlgebra  # Compute eigenvalues.
using ProgressMeter  # Show progress during 'for' loops.
using Plots


# Generate matrix from ensemble.
function make_matrix(N, c; μ=0, σ=1/sqrt(c))
    
    graph = random_regular_graph(N, c)
    A = adjacency_matrix(graph)
    
    d = Normal(μ, σ)
    J = Symmetric(rand(d, (N, N)))
    
    M = Symmetric(A .* J)
    
    return M
end


# Create arrays of M2 values for the solution of the cavity and marginal precisions.
function make_recursion_arrays(M)
    
    c = sum(M[1, :] .!= 0.)
    N = size(M)[1]
    
    M2 = Symmetric(M .* M)  # Element-wise-squared.
    
    cavity_m = zeros(c-1, N*c)
    margin_m = zeros(c, N)
    
    pos_list = []  # Positions of cavity precisions in array.
    
    col = 1  # Counter for columns of cavity_m.
    
    for j ∈ 1:N
        
        j_neigh = [k for k ∈ 1:N if M[j, k] != 0]
        margin_m[:, j] .= M2[j_neigh, j]
        
        for k ∈ j_neigh
            knj_neigh = [l for l in 1:N if M[k, l] != 0 && l != j]
            cavity_m[:, col] .= M2[knj_neigh, k]
            ω_positions = [(k-1)*c + sum(M[1:l,k] .!= 0) for l in knj_neigh]
            
            push!(pos_list, ω_positions)
            
            col += 1
        end
    end
    
    return cavity_m, margin_m, pos_list
end


function update_precisions!(ω, z, cavity_m, pos_list)
    Δω_max = 0
            
    for i in 1:N*c
        ω_old = ω[i]  # Save value we're updating
        ids = pos_list[i]

        ω[i] = z + sum(cavity_m[:, i] ./ ω[ids])

        Δω = abs(ω[i] - ω_old)
        (Δω > Δω_max) && (Δω_max = Δω)
    end
    
    return Δω_max
end


function cavity_method(M, ϵ, λ_range; tolerance=1e-6, max_iter=1000)
    
    c = sum(M[1, :] .!= 0.)  # Mean connectivity.
    N = size(M)[1]  # Linear size of the matrix M.
    
    points = length(λ_range)
    
    cavity_m, margin_m, pos_list = make_recursion_arrays(M)
    
    ρ_values = zeros(points)
    
    # Initial values for the cavity precisions.
    ω = rand(Complex{Float64}, N*c)
    
    @showprogress for k ∈ 1:points
        
        λ = λ_range[k]
        z = im*(λ - im*ϵ)
        
        # Attempt to reach fixed point for cavity precisions.
        for iteration in 1:max_iter
            
            Δω_max = update_precisions!(ω, z, cavity_m, pos_list)
            (Δω_max < tolerance) && break
            (iteration == max_iter) && println("Exhausted iterations at $(λ).")
        end
        
        # Compute the marginal precisions.
        cavity_ω = reshape(ω, (c, N))
        margin_ω = z .+ sum(margin_m ./ cavity_ω, dims=1)
        
        # Compute ρ(λ)
        ρ = imag(sum(im ./ margin_ω)) / (π * N)
        ρ_values[k] = ρ
    end
    
    return ρ_values
end


function diago_vs_cavity(N, c, ϵ, λ_range, samples; μ=0, σ=1/sqrt(c), tolerance=1e-6, max_iter=1000)
    
    λ_values = []
    ρ_values = 0
    
    for i ∈ 1:samples
        M = make_matrix(N, c, μ=μ, σ=σ)
        
        append!(λ_values, eigvals(M))
        
        if i != 1
            ρ_values += cavity_method(M, ϵ, λ_range, tolerance=tolerance, max_iter=max_iter)
            continue
        end
        
        ρ_values = cavity_method(M, ϵ, λ_range, tolerance=tolerance, max_iter=max_iter)
        
    end
    
    ρ_values = ρ_values/samples
    
    return λ_values, ρ_values
end


# Function to create LaTeX ticks based on the maximum values of the plot.
function make_latex_ticks(plot)
    
    # Find maximum absolute values for x and y in the plot. 
    
    x_max = 0
    y_max = 0

    for series in plot.series_list
        x_values = series.plotattributes[:x]
        x_maxval = maximum([x for x in x_values if !isnan(x)])
        (x_maxval > x_max) && (x_max = x_maxval)

        y_values = series.plotattributes[:y]
        y_maxval = maximum([y for y in y_values if !isnan(y)])
        (y_maxval > y_max) && (y_max = y_maxval)
    end
    
    # Make xticks. ---------------------------------------------

    xticks_max = round(Int, x_max)
    xticks_range = -xticks_max:xticks_max
    xticks_label = ["\$ $(i) \$" for i ∈ xticks_range];
    xticks = (xticks_range, xticks_label)
        
    # Make yticks. ---------------------------------------------

    max_yticks = 6  # Avoid saturating with many ticks.
    
    if y_max < max_yticks * 0.1
        yticks_range = [y for y in 0:0.1:1 if y <= 1.05*y_max]
    else
        yticks_range = range(0, y_max, length=max_yticks)
    end

    yticks_label = ["\$ $(trunc(y, digits=2)) \$" for y in yticks_range]
    yticks = (yticks_range, yticks_label)
    
    return xticks, yticks
end

## 2. Generate data

In [ ]:
Random.seed!(42)  # Fix the seed of every call to random numbers.

N = 2^11
c = 3
ϵ = 0.005

samples = 1

λ_min = ϵ
λ_max = 3.
points = 100

λ_range = range(λ_min, λ_max, length=points)

λ_vals, ρ_vals = diago_vs_cavity(N, c, ϵ, λ_range, samples, tolerance=1e-7);

## 3. Make plot

In [ ]:
save_figure = true

##############################
# Prepare data for plotting. #
##############################

# Make binning for the histogram by centering
# the bars at the cavity-method values of λ.

Δλ = λ_range.step.hi
bin_λ = [-λ - (Δλ/2) for λ in reverse(λ_range)]
append!(bin_λ, -reverse(bin_λ))


# Complete spectral density values of the cavity 
# method for negative λ by using ρ(-λ) = ρ(λ).

λ_xplot = [-λ for λ in reverse(λ_range)]
append!(λ_xplot, λ_range)

ρ_yplot = reverse(ρ_vals)
append!(ρ_yplot, ρ_vals)


####################
# Initialize plot. #
####################

cavity_plot = histogram(λ_vals, label="Direct diagonalization",
                        normalize=:pdf, 
                        bins=bin_λ, 
                        alpha=0.45, 
                        color=:aqua)

# ρ(λ) via cavity method.
plot!(λ_xplot, ρ_yplot, label="Cavity method", color=:blue, lw=1.7)


###########################
# General plot attributes #
###########################

kw = (; 
        xlabel="\$ \\lambda \$",
        xminorgrid=true,
        xminorticks=2,
        xlabelfontsize=18,
    
        ylabel="\$ \\rho(\\lambda) \$",
        yminorgrid=true,
        ylabelfontsize=18,
    
        tickfontsize=18,
        legendfontsize=11,
    
        framestyle=:box,
        size=(600, 400),
        dpi=400)

plot!(;kw...)

xticks, yticks = make_latex_ticks(cavity_plot)

plot!(xticks=xticks, yticks=yticks, yminorticks=4)

if ! save_figure
    cavity_plot
else
    params_end = "_N_$(N)_c_$(c)_eps_$(ϵ)_samples_$(samples).png"
    savefig(cavity_plot, "./Figures/2_2_cavity" * params_end)
end